**Install dependencies and imports**

In [112]:
# !pip install hazm

In [113]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from hazm import Normalizer, word_tokenize, Stemmer, stopwords_list
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import re

**Connect to google drive**

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Read train data and Perform EDA (see features and drop nulls)**

In [115]:
df_train = pd.read_csv('/content/drive/MyDrive/UNI-4022/AI/train_data.csv')
df_train.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,استانها
1,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,استانها
2,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,استانها
3,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,اجتماعی
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,اقتصادی


In [116]:
df_train.describe()

,title,description,tags
count,12457,12457,12455
unique,12449,12324,100
top,عراقچی برای شرکت در نشست کمیسیون مشترک برجام و...,گروه استان‌ها ــ این روزها همه‌جا صحبت از جولا...,بایگانی
freq,2,3,1944


In [117]:
df_train.dropna(inplace=True)
df_train.describe()

,title,description,tags
count,12455,12455,12455
unique,12447,12322,100
top,نخست وزیر پیشین فرانسه به پنج سال زندان محکوم شد,رییس‌جمهور گفت: امسال سال سختی است؛ هم با ویرو...,بایگانی
freq,2,3,1944


**Filter correct tags and convert text tags to numerical classes**

In [118]:
print(df_train['tags'].unique())

['استانها' 'اجتماعی' 'اقتصادی' 'سیاسی' 'فرهنگی' 'بین الملل' 'ورزشی'
 'رسانه ها' 'بازار' 'الشرق الأوسط' 'دانش و محیط زیست' 'جهان' 'ایران'
 'NRS-Import' 'فرهنگ و هنر' 'آلمان' 'دیدگاه' 'دوره\u200cهای زبان آلمانی'
 'ورزش' 'اقتصاد' 'گوناگون' 'شبکه\u200cهای اجتماعی' 'دویچه وله'
 'Community D' 'آلمانی پیش\u200cرفته' 'ویدئو > گزارش' 'استان ها > مرکزی'
 'استان ها > قم' 'علمی و دانشگاهی > صنفی، فرهنگی'
 'اقتصادی > عمران و اشتغال' 'اقتصادی > تولید و تجارت' 'اجتماعی > سلامت'
 'اجتماعی > آموزش و پرورش' 'سیاسی > دولت'
 'فرهنگی و هنری > گردشگری و میراث' 'اقتصادی > اقتصاد کلان'
 'ورزشی > فوتبال، فوتسال' 'سیاسی > سیاست خارجی' 'اجتماعی > حوادث، انتظامی'
 'سیاسی > مجلس' 'اجتماعی > محیط زیست' 'سیاسی > اندیشه امام و رهبری'
 'اجتماعی > خانواده' 'سیاسی > حقوقی و قضایی' 'سیاسی > سیاست داخلی'
 'استان ها > لرستان' 'علمی و دانشگاهی > علم و فناوری ایران'
 'استان ها > گیلان' 'اجتماعی > جامعه، شهری' 'سیاسی > دفاعی - امنيتی'
 'علمی و دانشگاهی > پژوهش' 'گرافیک > اینفوگرافیک' 'استان ها > خوزستان'
 'اقتصادی > انرژی' 'ف

save 5 main classes

In [119]:
main_tags = {"علمی":5 ,"فرهنگی":5, "ورزشی":5, "سیاسی":4 , "بین الملل":3,"استانها":2,"اقتصادی":1, "اجتماعی":0, "استان ها":2}
updated_main_tags = {"علمی":5 ,"فرهنگی":5, "ورزشی":5, "سیاسی":4 , "بین الملل":3,"استانها":2,"اقتصادی":1, "اجتماعی":0, "استان ها":2}

Save other tags wich contain main classes words

In [120]:
unique_tags = df_train['tags'].unique()

# list to save tags wich does not contain main class words
unresolved_tags = []
for tag in unique_tags:
  for main_tag in main_tags.keys():
    if main_tag in tag:
      # save to dict to have numerical class
      updated_main_tags[tag] = main_tags[main_tag]
      break
  else:
    unresolved_tags.append(tag)

print(set(unresolved_tags))

{'Community D', 'اقتصاد ایران', 'عکس > خبری', 'ایسنا+ > ایسنا+', 'آلمان', 'آلمانی پیش\u200cرفته', 'الشرق الأوسط', 'دانش و فناوری', 'ايران', 'ویدئو > گزارش', 'گوناگون', 'بازار', 'NRS-Import', 'دویچه وله', 'فرهنگ و هنر', 'خبرخوان', 'دانش و محیط زیست', 'ورزش', 'ایران', 'بایگانی', 'رسانه ها', 'جهان', 'اقتصاد', 'گرافیک > اینفوگرافیک', 'دوره\u200cهای زبان آلمانی', 'دیدگاه'}


Save remainder tags wich can be converted to main classes using same method as before

In [121]:
new_tags = {'آلمانی': 3, 'آلمان': 3, 'دانش': 5, 'جهان':3, 'فرهنگ': 5, 'اقتصاد': 1, 'ورزش': 5, 'ایران': 2, 'ايران':2}

unique_tags = df_train['tags'].unique()
for tag in unique_tags:
  for main_tag in new_tags.keys():
    if main_tag in tag:
      updated_main_tags[tag] = new_tags[main_tag]
      # update unresolved tags
      if tag in unresolved_tags:
        unresolved_tags.remove(tag)

print(set(unresolved_tags))

{'Community D', 'گوناگون', 'عکس > خبری', 'ایسنا+ > ایسنا+', 'بازار', 'NRS-Import', 'دویچه وله', 'الشرق الأوسط', 'گرافیک > اینفوگرافیک', 'خبرخوان', 'بایگانی', 'رسانه ها', 'دیدگاه', 'ویدئو > گزارش'}


In [122]:
# replace textual tags values with specified numbers
df_train['tags'] = df_train['tags'].replace(updated_main_tags)

In [123]:
# counts of unique values of all tags
print(df_train['tags'].value_counts())

tags
3                       3037
4                       2717
بایگانی                 1944
2                       1863
0                       1159
1                        883
5                        757
دیدگاه                    31
خبرخوان                   20
NRS-Import                14
عکس > خبری                 8
رسانه ها                   7
گرافیک > اینفوگرافیک       5
ایسنا+ > ایسنا+            3
گوناگون                    2
الشرق الأوسط               1
بازار                      1
دویچه وله                  1
Community D                1
ویدئو > گزارش              1
Name: count, dtype: int64


Delete non unresolved tags (unnumerical tags)

In [124]:
# convert tags to numeric(NaN if can not be cast)
df_train['tags'] = pd.to_numeric(df_train['tags'], errors='coerce')

# removes rows containg NaN
df_train.dropna(subset=['tags'], inplace=True)

In [125]:
# check tags values
print(df_train['tags'].unique())

[2. 0. 1. 4. 5. 3.]


In [126]:
df_train.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,2.0
1,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,2.0
2,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,2.0
3,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,0.0
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,1.0


**Text preprocessing with hazm**

In [127]:
# Objects
normalizer = Normalizer()
stemmer = Stemmer()
stopwords = stopwords_list()

def preprocess(text):
    # remove numbers and hyperlinks
    text = re.sub(r'[\.|\+|0-9]','',text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)

    text = normalizer.normalize(text)
    tokens = word_tokenize(text)
    # Remove tokens if they are one of stop words
    tokens = [stemmer.stem(token) for token in tokens if token not in stopwords]

    return " ".join(tokens)

In [128]:
# call preprocessing for text columns of train data
df_train['title'] = df_train['title'].apply(preprocess)
df_train['description'] = df_train['description'].apply(preprocess)

In [129]:
# Combine two fields into one (for text vectorizing)
df_train['news'] = df_train['title'] + ' ' + df_train['description']

**Split the data**

In [130]:
# split data
X_train, X_validate, y_train, y_validate = train_test_split(df_train['news'], df_train['tags'], test_size=0.2, random_state=42)

**Vectorize data to have vectors instead of texts**

In [131]:
# Create an instance of the CountVectorizer or TfidfVectorizer and fit ti tarin
vectorizer = TfidfVectorizer()
vectorizer.fit(X_train)

TfidfVectorizer()

In [132]:
# transform the X_rain and X_validate into a vector(matrix)
X_train_vec = vectorizer.transform(X_train)
X_validate_vec = vectorizer.transform(X_validate)

In [133]:
# Train the SVM model instance
svm_model = SVC(kernel='rbf', C=15)  # C: Regularization parameter
svm_model.fit(X_train_vec, y_train)

SVC(C=15)

In [134]:
# Predict valiadet tags using the trained SVC model
y_validate_pred = svm_model.predict(X_validate_vec)

# show prediction report
print(classification_report(y_validate, y_validate_pred))

              precision    recall  f1-score   support

         0.0       0.78      0.81      0.80       208
         1.0       0.80      0.83      0.81       182
         2.0       0.90      0.80      0.85       351
         3.0       0.87      0.89      0.88       631
         4.0       0.84      0.90      0.87       554
         5.0       0.82      0.68      0.75       158

    accuracy                           0.85      2084
   macro avg       0.84      0.82      0.83      2084
weighted avg       0.85      0.85      0.85      2084



**Read test data and perform preprocessing and vectorization**

In [135]:
df_test = pd.read_csv('/content/drive/MyDrive/UNI-4022/AI/test_data.csv')
df_test.head()

,title,description
0,درگیر شایعات نشوید؛ نتیجه آزمایش افراد مشکوک ب...,گروه استان‌ها- رئیس دانشگاه علوم پزشکی استان ا...
1,تخریب منازل «علی‌آباد» خرم‌آباد در پی بارش بار...,گروه استان‌ها ـبارش‌های سیل‌آسا که از روز گذشت...
2,‌اختصاصی| قرنطینه آیت‌الله شبیری‌زنجانی صحت ند...,گروه استان‌ها ـ مسئول دفتر آیت الله شبیری زنجا...
3,احتمال طغیان رودخانه‌های غرب کشور/ کاهش شدت با...,رئیس سازمان هواشناسی از احتمال طغیان اغلب رودخ...
4,تحویل طومار مردم تهران برای پیگرد عاملان ترور ...,طومار امضا شده توسط هزاران تن از مردم تهران در...


In [136]:
# call preprocessing for text columns
df_test['title'] = df_test['title'].apply(preprocess)
df_test['description'] = df_test['description'].apply(preprocess)

# Combine the two fields into one
df_test['news'] = df_test['title'] + ' ' + df_test['description']

# transform the X_test into a vector(matrix)
X_test = vectorizer.transform(df_test['news'])

**Predict tags for test data and save it to CSV**

In [137]:
# Predict test tags using the trained SVC model
y_test_pred = svm_model.predict(X_test)

In [139]:
# Create a DataFrame with the predictions
results = pd.DataFrame({
    'prediction': y_test_pred
})

# Save the DataFrame to a CSV file
results.to_csv('/content/drive/MyDrive/UNI-4022/AI/submission.csv', index=False)